## Download Dataset 

Today we are using the open dataset from '今日頭條'(https://www.toutiao.com/)
There are 382688 titles along with their categories in this dataset. One article belongs to one category. Repository: https://github.com/skdjfla/toutiao-text-classfication-dataset

In [1]:
%%bash 
rm -rf /home/ec2-user/SageMaker/chinese-corpus 
cd /home/ec2-user/SageMaker/ 
mkdir chinese-corpus
cd chinese-corpus
git clone https://github.com/skdjfla/toutiao-text-classfication-dataset.git 
cp toutiao-text-classfication-dataset/toutiao_cat_data.txt.zip .
unzip toutiao_cat_data.txt.zip 


Archive:  toutiao_cat_data.txt.zip
  inflating: toutiao_cat_data.txt    


Cloning into 'toutiao-text-classfication-dataset'...


In [2]:
!head -n 10 /home/ec2-user/SageMaker/chinese-corpus/toutiao_cat_data.txt

6551700932705387022_!_101_!_news_culture_!_京城最值得你来场文化之旅的博物馆_!_保利集团,马未都,中国科学技术馆,博物馆,新中国
6552368441838272771_!_101_!_news_culture_!_发酵床的垫料种类有哪些？哪种更好？_!_
6552407965343678723_!_101_!_news_culture_!_上联：黄山黄河黄皮肤黄土高原。怎么对下联？_!_
6552332417753940238_!_101_!_news_culture_!_林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？_!_
6552475601595269390_!_101_!_news_culture_!_黄杨木是什么树？_!_
6552387648126714125_!_101_!_news_culture_!_上联：草根登上星光道，怎么对下联？_!_
6552271725814350087_!_101_!_news_culture_!_什么是超写实绘画？_!_
6552452982015787268_!_101_!_news_culture_!_松涛听雨莺婉转，下联？_!_
6552400379030536455_!_101_!_news_culture_!_上联：老子骑牛读书，下联怎么对？_!_
6552339283632455939_!_101_!_news_culture_!_上联：山水醉人何须酒。如何对下联？_!_


### Observe the number of documents over each category 

In [3]:
cat /home/ec2-user/SageMaker/chinese-corpus/toutiao_cat_data.txt | awk -F '_!_' '{print $3}' | sort | uniq -c | sort -k1n

    340 stock
   6273 news_story
  17672 news_house
  19322 news_agriculture
  21422 news_travel
  24984 news_military
  26909 news_world
  27058 news_edu
  27085 news_finance
  28031 news_culture
  29300 news_game
  35785 news_car
  37568 news_sports
  39396 news_entertainment
  41543 news_tech


### Check if there are multi-labeled documents 

In [4]:
cat /home/ec2-user/SageMaker/chinese-corpus/toutiao_cat_data.txt | awk -F '_!_' '{print $1}' | sort | uniq -c | sort -k1n | tail -10  

      1 6555273002437574919
      1 6555273170788549134
      1 6555273406252581128
      1 6555274004679098888
      1 6555275228337930510
      1 6555276205229097480
      1 6555276669471425038
      1 6555277029976048142
      1 6555284827275264519
      1 6555289097043181837


### Install necessary packages 

In [5]:
!pip install jieba
!pip install ckiptagger 
!pip install tensorflow==1.13.1

     |████████████████████████████████| 19.2 MB 5.6 MB/s eta 0:00:01
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314478 sha256=044f5758f9bf61b5b4c9a08214f80b7b4febb1634de506fdc0b28ba0047266ca
  Stored in directory: /home/ec2-user/.cache/pip/wheels/17/a7/8b/a7e03881534e78558920ac68aaeca05180c0e2c3d11c4fce3b
Successfully built jieba
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 92.5 MB 91 kB/s s eta 0:00:01
     |████████████████████████████████| 50 kB 2.1 MB/s  eta 0:00:01
     |████████████████████████████████| 3.0 MB 55.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 55.3 MB/s eta 0:00:01
     |████████████████████████████████| 367 kB 59.3 MB/s eta 0:00:01
     |███████████

In [ ]:
import sys
sys.path.append('/home/ec2-user/SageMaker/nlp_processing/')

from preprocess import TouTiaoNewsPreprocessor
from segmenter import JiebaSegmenter
from transformer import BlazingTextInputDataTransformer

preprocessor = TouTiaoNewsPreprocessor()
res = preprocessor.preprocess('/home/ec2-user/SageMaker/chinese-corpus/toutiao_cat_data.txt')
print('result of preprocess:', res[0:10])
segmenter = JiebaSegmenter()
transformer = BlazingTextInputDataTransformer()
all_contents = [] 
for r in res:
    toks = segmenter.segment(r[2])
    input = transformer.transform((r[0], r[1], toks))
    all_contents.append(input)    
print('result of transformed data:', all_contents[0:10])    

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/hom

result of preprocess: [('101', 'news_culture', '京城最值得你来场文化之旅的博物馆'), ('101', 'news_culture', '发酵床的垫料种类有哪些？哪种更好？'), ('101', 'news_culture', '上联：黄山黄河黄皮肤黄土高原。怎么对下联？'), ('101', 'news_culture', '林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？'), ('101', 'news_culture', '黄杨木是什么树？'), ('101', 'news_culture', '上联：草根登上星光道，怎么对下联？'), ('101', 'news_culture', '什么是超写实绘画？'), ('101', 'news_culture', '松涛听雨莺婉转，下联？'), ('101', 'news_culture', '上联：老子骑牛读书，下联怎么对？'), ('101', 'news_culture', '上联：山水醉人何须酒。如何对下联？')]


Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.942 seconds.
Prefix dict has been built successfully.


In [ ]:
import pickle 
pickle.dump( all_contents, open( "/home/ec2-user/SageMaker/chinese-corpus/all_contents.p", "wb" ) )